### 統計學習與深度學習
### Homework 5

### Q1

In [16]:
%matplotlib inline

import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import random

random.seed(1223)
labels = ['blazer', 'cardigan', 'coat', 'jacket']

pic_num_train = [0, 0, 0, 0]
pic_sum_train = 0

for i in range(4):
    basepath = os.path.join("photos/train/", labels[i])
    for root,dirs,files in os.walk(basepath):
        for each in files:
            pic_num_train[i] += 1
    
    pic_sum_train += pic_num_train[i]
print("Total amount of pictures in train=", pic_sum_train)
for i in range(4):
    print("amount of", labels[i], "in train =", pic_num_train[i])
    print("account for", "%.2f" %(pic_num_train[i]/pic_sum_train*100), "%")

Total amount of pictures in train= 1056
amount of blazer in train = 99
account for 9.38 %
amount of cardigan in train = 243
account for 23.01 %
amount of coat in train = 297
account for 28.12 %
amount of jacket in train = 417
account for 39.49 %


In [17]:
pic_num_valid = [0, 0, 0, 0]
pic_sum_valid = 0
for i in range(4):
    basepath = os.path.join("photos/valid/", labels[i])
    for root,dirs,files in os.walk(basepath):
        for each in files:
            pic_num_valid[i] += 1
    
    pic_sum_valid += pic_num_valid[i]
print("Total amount of pictures in valid =", pic_sum_valid)
for i in range(4):
    print("amount of", labels[i], "in valid =", pic_num_valid[i])
    print("account for", "%.2f" %(pic_num_valid[i]/pic_sum_valid*100), "%")

Total amount of pictures in valid = 105
amount of blazer in valid = 7
account for 6.67 %
amount of cardigan in valid = 36
account for 34.29 %
amount of coat in valid = 27
account for 25.71 %
amount of jacket in valid = 35
account for 33.33 %


In [18]:
pic_num_test = [0, 0, 0, 0]
pic_sum_test = 0
for i in range(4):
    basepath = os.path.join("photos/test/", labels[i])
    for root,dirs,files in os.walk(basepath):
        for each in files:
            pic_num_test[i] += 1
    
    pic_sum_test += pic_num_test[i]
print("Total amount of pictures in test =", pic_sum_test)
for i in range(4):
    print("amount of", labels[i], "in test =", pic_num_test[i])
    print("account for", "%.2f" %(pic_num_test[i]/pic_sum_test*100), "%")

Total amount of pictures in test = 146
amount of blazer in test = 9
account for 6.16 %
amount of cardigan in test = 42
account for 28.77 %
amount of coat in test = 43
account for 29.45 %
amount of jacket in test = 52
account for 35.62 %


我覺得blazer的準確率應該會最高，且明顯較高，因為樣式比較固定且材質特殊，容易捕捉特徵。
再來是jacket，雖然它的材質歧異度大，不易捕捉特徵，但跟剩下的兩種差異較明顯，所以準確率應該會略高於剩下兩個，位居第二。
cardigan跟coat的相似度頗高，而cardigan的材質歧異度又較大，不易捕捉特徵，可能被判斷成jacket或coat，所以我認為cardigan準確率應該會最低。
coat則是第三。

### Q3 (順序寫錯了，Q2在後面)

In [1]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time

resnet50 = models.resnet50(pretrained = True)

In [2]:
image_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomRotation(degrees=30),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

In [5]:
dataset = "photos"
train_basepath = os.path.join(dataset, "train")
valid_basepath = os.path.join(dataset, "valid")
test_basepath = os.path.join(dataset, "test")

IMAGE_SIZE = (224, 224)
NUM_EPOCHS = 200
NUM_CLASS = 4
BATCH_SIZE = 32

data = {
    'train': datasets.ImageFolder(root=train_basepath, transform=image_transforms['train']),
    
    'valid': datasets.ImageFolder(root=valid_basepath, transform=image_transforms['valid']),
    
    'test': datasets.ImageFolder(root=test_basepath, transform=image_transforms['test'])
}
 
 
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])

train_data = DataLoader(data['train'], batch_size=BATCH_SIZE, shuffle=True)
valid_data = DataLoader(data['valid'], batch_size=BATCH_SIZE, shuffle=True)
test_data = DataLoader(data['test'], batch_size=BATCH_SIZE, shuffle=False)

In [6]:
for param in resnet50.parameters():
    param.requires_grad = False

fc_inputs = resnet50.fc.in_features

resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 4),
    nn.LogSoftmax(dim=1)
)

In [138]:
loss_func = nn.NLLLoss()
optimizer = optim.Adam(resnet50.parameters())

In [145]:
def train_and_valid(model, loss_function, optimizer, epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    history = []
    best_acc = 0.0
    best_epoch = 0
    patience = 20
    trigger_times = 0
    last_avg_valid_loss = 100000
    
    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
 
        model.train()
 
        train_loss = 0.0
        train_acc = 0.0
        valid_loss = 0.0
        valid_acc = 0.0
 
        for i, (inputs, labels) in enumerate(train_data):
            inputs = inputs.to(device)
            labels = labels.to(device)
 
            optimizer.zero_grad()
 
            outputs = model(inputs)
 
            loss = loss_function(outputs, labels)
 
            loss.backward()
 
            optimizer.step()
 
            train_loss += loss.item() * inputs.size(0)
 
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
 
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
 
            train_acc += acc.item() * inputs.size(0)
 
        with torch.no_grad():
            model.eval()
 
            for j, (inputs, labels) in enumerate(valid_data):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
 
                loss = loss_function(outputs, labels)
 
                valid_loss += loss.item() * inputs.size(0)
 
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))
 
                acc = torch.mean(correct_counts.type(torch.FloatTensor))
 
                valid_acc += acc.item() * inputs.size(0)
 
        avg_train_loss = train_loss/train_data_size
        avg_train_acc = train_acc/train_data_size
 
        avg_valid_loss = valid_loss/valid_data_size
        avg_valid_acc = valid_acc/valid_data_size
        
        if avg_valid_loss > last_avg_valid_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                return model, history
        
        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
 
        if best_acc < avg_valid_acc:
            best_acc = avg_valid_acc
            best_epoch = epoch + 1
 
        epoch_end = time.time()
 
        print("Epoch: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
            epoch+1, avg_valid_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start
        ))
        print("Best Accuracy for validation : {:.4f} at epoch {:03d}".format(best_acc, best_epoch))
 
        torch.save(model, dataset+'_model_'+str(epoch+1)+'.pt')
    
        last_avg_valid_loss = avg_valid_loss
        
    return model, history

In [155]:
num_epochs = 200
trained_model, history = train_and_valid(resnet50, loss_func, optimizer, num_epochs)
torch.save(history, dataset+'_history.pt')

history = np.array(history)
plt.plot(history[:, 0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0, 1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()
 
plt.plot(history[:, 2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

Epoch: 1/200
Epoch: 001, Training: Loss: 0.9907, Accuracy: 65.4356%, 
		Validation: Loss: 0.9907, Accuracy: 63.8095%, Time: 189.1098s
Best Accuracy for validation : 0.6381 at epoch 001
Epoch: 2/200
Epoch: 002, Training: Loss: 0.9478, Accuracy: 64.6780%, 
		Validation: Loss: 0.9478, Accuracy: 62.8571%, Time: 187.1130s
Best Accuracy for validation : 0.6381 at epoch 001
Epoch: 3/200
trigger times: 1
Epoch: 003, Training: Loss: 1.0642, Accuracy: 65.8144%, 
		Validation: Loss: 1.0642, Accuracy: 59.0476%, Time: 190.6657s
Best Accuracy for validation : 0.6381 at epoch 001
Epoch: 4/200
Epoch: 004, Training: Loss: 0.9346, Accuracy: 62.7841%, 
		Validation: Loss: 0.9346, Accuracy: 61.9048%, Time: 189.8512s
Best Accuracy for validation : 0.6381 at epoch 001
Epoch: 5/200
Epoch: 005, Training: Loss: 0.9096, Accuracy: 67.3295%, 
		Validation: Loss: 0.9096, Accuracy: 56.1905%, Time: 198.8912s
Best Accuracy for validation : 0.6381 at epoch 001
Epoch: 6/200
trigger times: 2
Epoch: 006, Training: Loss: 

TypeError: cannot unpack non-iterable ResNet object

In [21]:
class_acc = [0, 0, 0, 0]
trained_model = torch.load('photos_model_33.pt')
loss_func = nn.NLLLoss()
optimizer = optim.Adam(resnet50.parameters())

test_loss = 0.0
test_acc = 0.0

for j, (inputs, labels) in enumerate(test_data):        
    outputs = trained_model(inputs)
 
    ret, predictions = torch.max(outputs.data, 1)
    correct_counts = predictions.eq(labels.data.view_as(predictions))
    for l in range(len(predictions)):
        if predictions[l] == labels[l]:
            if labels[l].eq(0):
                class_acc[0] +=1
            elif labels[l].eq(1):
                class_acc[1] +=1
            elif labels[l].eq(2):
                class_acc[2] +=1
            else :
                class_acc[3] +=1

                
    acc = torch.mean(correct_counts.type(torch.FloatTensor))
    test_acc += acc.item() * inputs.size(0)
 
    avg_test_acc = test_acc/test_data_size
    
        
class_acc[0] = class_acc[0]/pic_num_test[0]
class_acc[1] = class_acc[1]/pic_num_test[1]
class_acc[2] = class_acc[2]/pic_num_test[2]
class_acc[3] = class_acc[3]/pic_num_test[3]

print("acc =", "%.2f" %(avg_test_acc*100), "%")
print("blazer acc =", "%.2f" %(class_acc[0]*100), "%")
print("cardigan acc =", "%.2f" %(class_acc[1]*100), "%")
print("coat acc =", "%.2f" %(class_acc[2]*100), "%")
print("jacket acc =", "%.2f" %(class_acc[3]*100), "%")

acc = 63.01 %
blazer acc = 44.44 %
cardigan acc = 40.48 %
coat acc = 62.79 %
jacket acc = 84.62 %


### Q2

In [7]:
resnet50_2 = models.resnet50(pretrained = True)

In [8]:
for param in resnet50_2.parameters():
    param.requires_grad = True

fc_inputs = resnet50_2.fc.in_features

resnet50.fc = nn.Sequential(
    nn.Linear(fc_inputs, 4),
    nn.LogSoftmax(dim=1)
)

In [9]:
loss_func = nn.NLLLoss()
optimizer = optim.Adam(resnet50_2.parameters())

In [12]:
num_epochs = 200
trained_model2, history = train_and_valid(resnet50_2, loss_func, optimizer, num_epochs)

Epoch: 1/200


KeyboardInterrupt: 

In [19]:
class_acc = [0, 0, 0, 0]
trained_model = torch.load('photos_model2_39.pt')
loss_func = nn.NLLLoss()
optimizer = optim.Adam(resnet50.parameters())

test_loss = 0.0
test_acc = 0.0

for j, (inputs, labels) in enumerate(test_data):        
    outputs = trained_model(inputs)
 
    ret, predictions = torch.max(outputs.data, 1)
    correct_counts = predictions.eq(labels.data.view_as(predictions))
    for l in range(len(predictions)):
        if predictions[l] == labels[l]:
            if labels[l].eq(0):
                class_acc[0] +=1
            elif labels[l].eq(1):
                class_acc[1] +=1
            elif labels[l].eq(2):
                class_acc[2] +=1
            else :
                class_acc[3] +=1

                
    acc = torch.mean(correct_counts.type(torch.FloatTensor))
    test_acc += acc.item() * inputs.size(0)
 
    avg_test_acc = test_acc/test_data_size
    
        
class_acc[0] = class_acc[0]/pic_num_test[0]
class_acc[1] = class_acc[1]/pic_num_test[1]
class_acc[2] = class_acc[2]/pic_num_test[2]
class_acc[3] = class_acc[3]/pic_num_test[3]

print("acc =", "%.2f" %(avg_test_acc*100), "%")
print("blazer acc =", "%.2f" %(class_acc[0]*100), "%")
print("cardigan acc =", "%.2f" %(class_acc[1]*100), "%")
print("coat acc =", "%.2f" %(class_acc[2]*100), "%")
print("jacket acc =", "%.2f" %(class_acc[3]*100), "%")

acc = 63.70 %
blazer acc = 55.56 %
cardigan acc = 45.24 %
coat acc = 48.84 %
jacket acc = 92.31 %


我原本預期jacket會是第二，但預測jacket的準確率比我想像中高了很多，壓倒性的第一，而blazer則比我想像中低的多，我認為可能是因為電腦在影像辨識時，只看衣服佔圖片的長度，卻沒有辨識衣服在人身上的"相對長度"，再加上有些coat跟blazer看起來的材質相近，因此可能是因此常常把這兩者分類錯誤，才使得兩者的準確率都不高。
而cardigan則是和預期的一樣，較難準確預測。

### Q5

根據我的觀察，Q2的model的準確率比Q3的model好一些，我認為這是因為Q2的model會根據訓練狀況調整weight，提高效果較佳的layer的影響，使得在有early stopping的機制下，training時能夠抵達更深層，因此整體的預測效果會優於Q3在訓練過程中固定每層的weight的model，但Q2的model則需要花費更長的時間訓練。而Q4的model由於未使用預初始化好的模型，因此可能因未初始化或初始化不當導致梯度收斂狀況較差，因此結果不一定比Q2好。